In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df_plan_metro = pd.read_csv('/content/plan du métro.csv')

In [ ]:
df_position_gps = pd.read_csv('/content/position gps des stations de métro.csv')

In [ ]:
df_passagers = pd.read_csv('/content/passagers.csv')

#Traitement des données

On en ressort le dataframe *trajets* avec les passagers qui vont d'une gare A à un gare B avec leurs coordonnées GPS.

##Position GPS des gares

In [ ]:
position_gps = df_position_gps.copy()

In [ ]:
position_gps['latitude'] = df_position_gps['GPS'].str.split(',').str[0]
position_gps['longitude'] = df_position_gps['GPS'].str.split(',').str[1]
position_gps.drop('GPS', axis=1, inplace=True)

In [ ]:
position_gps = position_gps.drop_duplicates(subset="Station", keep="first")

##Ajout des données gps

###Illustartion du manque de données gps pour certaines stations

In [ ]:
trajets = df_passagers.copy()

In [ ]:
trajets.to_csv('/content/trajets.csv', index=False)

In [ ]:
trajets = pd.merge(trajets, position_gps, left_on="de", right_on="Station", how="left")
trajets.rename(columns={'latitude': 'latitude_depart', 'longitude': 'longitude_depart'}, inplace=True)
trajets.drop(columns=["Station"], inplace=True)

trajets = pd.merge(trajets, position_gps, left_on="vers", right_on="Station", how="left")
trajets.rename(columns={'latitude': 'latitude_arrivee', 'longitude': 'longitude_arrivee'}, inplace=True)
trajets.drop(columns=["Station"], inplace=True)
trajets

,de,vers,nombre,latitude_depart,longitude_depart,latitude_arrivee,longitude_arrivee
0,La Défense,Esplanade de La Défense,7,48.89218707644951,2.237018056395014,48.88813847761218,2.2497927690097654
1,La Défense,Pont de Neuilly,5,48.89218707644951,2.237018056395014,48.88470820133096,2.260515079251701
2,La Défense,Les Sablons,6,48.89218707644951,2.237018056395014,48.88101649749112,2.272238940611057
3,La Défense,Porte Maillot,6,48.89218707644951,2.237018056395014,48.87793813782425,2.283913993903496
4,La Défense,Argentine,5,48.89218707644951,2.237018056395014,48.87533684211653,2.2901275088247015
...,...,...,...,...,...,...,...
93937,Olympiades,Châtillon - Montrouge,8,48.82727083601133,2.3680326484142777,NaN,NaN
93938,Olympiades,Saint-Ouen,4,48.82727083601133,2.3680326484142777,48.9045663899011,2.3226263683342268
93939,Olympiades,Pont Cardinet,6,48.82727083601133,2.3680326484142777,48.88956082831538,2.3151768973047844
93940,Olympiades,Cour Saint-Émilion,5,48.82727083601133,2.3680326484142777,48.83333855925167,2.386632402066985


In [ ]:
print('On a les coord gps de ' +str(position_gps.groupby('Station').count().shape[0]) + ' stations')

On a les coord gps de 261 stations


In [ ]:
print('Or il y a en tout ' + str(trajets.groupby('vers').count().shape[0]) + ' stations')

Or il y a en tout 307 stations


In [ ]:
absentes = set(trajets["de"]) - set(position_gps["Station"])
len(list(absentes))

46

In [ ]:
absentes = list(absentes)

In [ ]:
stations_trajets = trajets.groupby('vers').count()
stations_gps = position_gps.groupby('Station').count()
stations_absentes = trajets[~trajets["de"].isin(position_gps["Station"])]

In [ ]:
stations_absentes.groupby('de').count().shape[0]

46

In [ ]:
stations_trajets

,de,nombre,latitude_depart,longitude_depart,latitude_arrivee,longitude_arrivee
vers,,,,,,
Abbesses,306,306,260,260,306,306
Aimé Césaire,306,306,260,260,306,306
Alexandre Dumas,306,306,260,260,306,306
Alma - Marceau,306,306,260,260,306,306
Alésia,306,306,260,260,306,306
...,...,...,...,...,...,...
École Militaire,306,306,260,260,306,306
École vétérinaire de Maisons-Alfort,306,306,261,261,0,0
Église d'Auteuil,306,306,260,260,306,306


###Création du dataset de coord gps des stations complet

In [ ]:
df_merged = pd.merge(trajets, position_gps, left_on="de", right_on="Station", how="left")

In [ ]:
df_merged.rename(columns={'latitude': 'latitude_depart', 'longitude': 'longitude_depart'}, inplace=True)

In [ ]:
df_merged.drop(columns=["Station"], inplace=True)

In [ ]:
df_merged

,de,vers,nombre,latitude_depart,longitude_depart,latitude_arrivee,longitude_arrivee,latitude_depart,longitude_depart
0,La Défense,Esplanade de La Défense,7,48.89218707644951,2.237018056395014,48.88813847761218,2.2497927690097654,48.89218707644951,2.237018056395014
1,La Défense,Pont de Neuilly,5,48.89218707644951,2.237018056395014,48.88470820133096,2.260515079251701,48.89218707644951,2.237018056395014
2,La Défense,Les Sablons,6,48.89218707644951,2.237018056395014,48.88101649749112,2.272238940611057,48.89218707644951,2.237018056395014
3,La Défense,Porte Maillot,6,48.89218707644951,2.237018056395014,48.87793813782425,2.283913993903496,48.89218707644951,2.237018056395014
4,La Défense,Argentine,5,48.89218707644951,2.237018056395014,48.87533684211653,2.2901275088247015,48.89218707644951,2.237018056395014
...,...,...,...,...,...,...,...,...,...
93937,Olympiades,Châtillon - Montrouge,8,48.82727083601133,2.3680326484142777,NaN,NaN,48.82727083601133,2.3680326484142777
93938,Olympiades,Saint-Ouen,4,48.82727083601133,2.3680326484142777,48.9045663899011,2.3226263683342268,48.82727083601133,2.3680326484142777
93939,Olympiades,Pont Cardinet,6,48.82727083601133,2.3680326484142777,48.88956082831538,2.3151768973047844,48.82727083601133,2.3680326484142777
93940,Olympiades,Cour Saint-Émilion,5,48.82727083601133,2.3680326484142777,48.83333855925167,2.386632402066985,48.82727083601133,2.3680326484142777


In [ ]:
from math import sqrt

In [ ]:
df_sup_stations = pd.read_csv('/content/sup_stations.csv', sep=';')

In [ ]:
sup_stations = df_sup_stations.copy()

In [ ]:
sup_stations.drop('ligne', axis=1, inplace=True)

In [ ]:
station_coords = pd.concat([position_gps, sup_stations], ignore_index=True)

In [ ]:
station_coords['latitude'] = station_coords['latitude'].astype(float)
station_coords['longitude'] = station_coords['longitude'].astype(float)

In [ ]:
station_coords

,Station,latitude,longitude
0,Boucicaut,48.841094,2.287946
1,Bourse,48.868654,2.341376
2,Bir-Hakeim,48.853943,2.289335
3,Argentine,48.875337,2.290128
4,Daumesnil,48.839550,2.395703
...,...,...,...
302,Sully - Morland,48.851271,2.361853
303,Barbès - Rochechouart,48.883776,2.350607
304,Pré-Saint-Gervais,48.880160,2.398581
305,Quatre-Septembre,48.869659,2.336319


In [ ]:
station_coords.dtypes

,0
Station,object
latitude,float64
longitude,float64


### Ajout des coords gps aux trajets

In [ ]:
trajets = df_passagers.copy()

trajets = pd.merge(trajets, station_coords, left_on="de", right_on="Station", how="left")
trajets.rename(columns={'latitude': 'latitude_depart', 'longitude': 'longitude_depart'}, inplace=True)
trajets.drop(columns=["Station"], inplace=True)

trajets = pd.merge(trajets, station_coords, left_on="vers", right_on="Station", how="left")
trajets.rename(columns={'latitude': 'latitude_arrivee', 'longitude': 'longitude_arrivee'}, inplace=True)
trajets.drop(columns=["Station"], inplace=True)

trajets.rename(columns={'de': 'station_depart', 'vers': 'station_arrivee'}, inplace=True)

In [ ]:
trajets

,station_depart,station_arrivee,nombre,latitude_depart,longitude_depart,latitude_arrivee,longitude_arrivee
0,La Défense,Esplanade de La Défense,7,48.892187,2.237018,48.888138,2.249793
1,La Défense,Pont de Neuilly,5,48.892187,2.237018,48.884708,2.260515
2,La Défense,Les Sablons,6,48.892187,2.237018,48.881016,2.272239
3,La Défense,Porte Maillot,6,48.892187,2.237018,48.877938,2.283914
4,La Défense,Argentine,5,48.892187,2.237018,48.875337,2.290128
...,...,...,...,...,...,...,...
93937,Olympiades,Châtillon - Montrouge,8,48.827271,2.368033,48.810751,2.301718
93938,Olympiades,Saint-Ouen,4,48.827271,2.368033,48.904566,2.322626
93939,Olympiades,Pont Cardinet,6,48.827271,2.368033,48.889561,2.315177
93940,Olympiades,Cour Saint-Émilion,5,48.827271,2.368033,48.833339,2.386632


In [ ]:
#Vérification des valeurs manquantes
lignes_nan = trajets[trajets.isna().any(axis=1)]
lignes_nan

,station_depart,station_arrivee,nombre,latitude_depart,longitude_depart,latitude_arrivee,longitude_arrivee


# Trouver le trajet optimal entre deux gares

A ce stade on a:
- station_coords: dataframe (station, latitude, longitude)
- trajets: dataframe (passagers, station de départ, station d'arrivée avec leur coords)
- df_plan_metro: dataframe(connexion entre chaque gare et correspondance) pas encore utilisé mais il faudrait faire une "matrice d'adjacence"

In [ ]:
def get_index(station_name):
  return station_coords[station_coords['Station'] == station_name].index[0]

def get_station_name(index):
  return station_coords.iloc[index,0]

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    return sqrt((lat1-lat2)**2 + (lon1-lon2)**2)

def distance(station_index1, station_index2):
  lat1 = station_coords.iloc[station_index1,1]
  lon1 = station_coords.iloc[station_index1,2]
  lat2 = station_coords.iloc[station_index2,1]
  lon2 = station_coords.iloc[station_index2,2]
  return sqrt((lat1-lat2)**2 + (lon1-lon2)**2)

Distance matrix:

In [ ]:
n = station_coords.shape[0]
distance_matrix = np.zeros((n,n))
for i in range(n):
  for j in range(n):
    distance_matrix[i,j] = distance(i,j)

Adjency matrix:

In [ ]:
df_plan_metro

,de Ligne,de Station,vers Ligne,vers Station
0,1,Charles de Gaulle - Étoile,2,Charles de Gaulle - Étoile
1,1,Charles de Gaulle - Étoile,6,Charles de Gaulle - Étoile
2,1,Franklin D. Roosevelt,9,Franklin D. Roosevelt
3,1,Champs-Élysées - Clemenceau,13,Champs-Élysées - Clemenceau
4,1,Concorde,8,Concorde
...,...,...,...,...
1005,14,Châtelet,14,Gare de Lyon
1006,14,Gare de Lyon,14,Bercy
1007,14,Bercy,14,Cour Saint-Émilion
1008,14,Cour Saint-Émilion,14,Bibliothèque François-Mitterrand


In [ ]:
station_connexion = df_plan_metro[df_plan_metro['de Station'] != df_plan_metro['vers Station']]

In [ ]:
station_connexion.shape

(778, 4)

In [ ]:
m = station_connexion.shape[0]
adjency_matrix = np.zeros((m,m))
for i in range(m):
  index_depart = get_index(station_connexion.iloc[i,1])
  index_arrivee = get_index(station_connexion.iloc[i,3])
  adjency_matrix[index_depart,index_arrivee] = 1
  adjency_matrix[index_arrivee,index_depart] = 1

In [ ]:
def get_adjacent_stations(station1, traveled_distance, station2, route):
  """
  station1: int, departure station index
  traveled_distance: float, distance parcourue pour arriver à station1
  station2: int, arrival station index
  route: list, stations visitées pour arriver à station1
  """

  adjacent_stations = pd.DataFrame(columns=['station', 'traveled_distance','distance_to_station2','cout','route','sation_name'])


  for i in range(n):
    #Select the stations adjacent to station1
    if adjency_matrix[station1,i] != 0:
      #Compute distances from station1 to station i and station2
      distance_to_station2 = distance_matrix[i,station2]
      new_traveled_distance = traveled_distance + distance_matrix[station1,i]

      #Add an extra cost if there is a line change PAS ENCORE AU POINT
      if adjency_matrix[station1,i] == 2:
        new_traveled_distance += 100

      #Add the station to the route
      route_i = route.copy()
      route_i.append(i)

      #Add station i and his parameters to potential next stations to explore
      adjacent_stations = pd.concat([adjacent_stations,
                                     pd.DataFrame({'station': [i],
                                                   'traveled_distance': [new_traveled_distance.astype(float)],
                                                   'distance_to_station2': [distance_to_station2.astype(float)],
                                                   'cout': [(new_traveled_distance + distance_to_station2).astype(float)],
                                                   'route': [route_i],
                                                   'sation_name': [get_station_name(i)]
                                                   })], ignore_index=True)

  return adjacent_stations

In [ ]:
def trouver_trajets(station1, station2, traveled_distance, route, next_stations):
  """
  station1: int, departure station index
  station2: int, arrival station index
  traveled_distance: float, distance traveled to reach station1
  route: list, stations already seen
  next_stations: dataframe, stations to visit with the traveled distance to reach them
  """
  print("station1: " + get_station_name(station1))

  adjacent_stations = get_adjacent_stations(station1, traveled_distance, station2, route)

  #add the adjacent stations to station to visit
  new_next_stations = pd.concat([next_stations, adjacent_stations], ignore_index=True).sort_values(by='cout',ascending=True,ignore_index=True).drop_duplicates('station', keep="first")
  #compute the station that bring the closest to station2
  new_station1= new_next_stations.iloc[0,0]
  new_traveled_distance = new_next_stations.iloc[0,1]
  new_route = new_next_stations.iloc[0,4]

  #drop the new station from next stations possible
  new_next_stations = new_next_stations.drop(index = new_next_stations[new_next_stations['station'] == new_station1].index) #maybe just delete index0

  if new_station1 == station2:
    return route

  else:
    return trouver_trajets(new_station1, station2, new_traveled_distance, new_route, new_next_stations)

In [ ]:
next_stations_init = pd.DataFrame(columns=['station', 'traveled_distance','distance_to_station2', 'cout'])
station_index1 = get_index('Bérault')
station_index2 = get_index('Château de Vincennes')

In [ ]:
distance_matrix[station_index1,station_index2]

0.011280506417420172

In [ ]:
get_index('Saint-Mandé')

101

In [ ]:
print(station_index1)
print(station_index2)

182
226


In [ ]:
route1 = trouver_trajets(station_index1, get_index('Champs-Élysées - Clemenceau'),0,[station_index1],next_stations_init)

station1: Bérault
station1: Saint-Mandé
station1: Porte de Vincennes
station1: Nation
station1: Rue des Boulets
station1: Charonne
station1: Reuilly - Diderot
station1: Voltaire
station1: Faidherbe - Chaligny
station1: Ledru-Rollin
station1: Bastille
station1: Saint-Ambroise
station1: Saint-Paul
station1: Hôtel de Ville
station1: Châtelet
station1: Louvre - Rivoli
station1: Palais-Royal - Musée du Louvre
station1: Tuileries
station1: Concorde


In [ ]:
route1 = trouver_trajets(station_index1, get_index('Champs-Élysées - Clemenceau'),0,[station_index1],next_stations_init)

station1: Bérault
station1: Saint-Mandé
station1: Porte de Vincennes
station1: Nation
station1: Rue des Boulets
station1: Charonne
station1: Reuilly - Diderot
station1: Voltaire
station1: Faidherbe - Chaligny
station1: Ledru-Rollin
station1: Bastille
station1: Saint-Ambroise
station1: Saint-Paul
station1: Hôtel de Ville
station1: Châtelet
station1: Louvre - Rivoli
station1: Palais-Royal - Musée du Louvre
station1: Tuileries
station1: Concorde


In [ ]:
route1

[182, 101, 45, 67, 274, 277, 290, 56, 22, 228, 54, 292, 298, 207, 75]

In [ ]:
for i in range(len(route1)):
  print(get_station_name(route1[i]))

Bérault
Saint-Mandé
Porte de Vincennes
Nation
Reuilly - Diderot
Faidherbe - Chaligny
Ledru-Rollin
Bastille
Saint-Paul
Hôtel de Ville
Châtelet
Louvre - Rivoli
Palais-Royal - Musée du Louvre
Tuileries
Concorde


In [ ]:
trouver_trajets(station_index1,station_index2,0.,[],next_stations_init)

station1: Bérault
  station  traveled_distance  distance_to_station2      cout
1     226           0.011281              0.000000  0.011281
0     101           0.010489              0.021769  0.032258
station                  object
traveled_distance       float64
distance_to_station2    float64
cout                    float64
dtype: object


[226]

In [ ]:
trouver_trajets(station_index1,station_index2,0,[],next_stations_init)

station1: Bérault
  station  traveled_distance  distance_to_station2      cout
1     226           0.011281              0.000000  0.011281
0     101           0.010489              0.021769  0.032258
station                  object
traveled_distance       float64
distance_to_station2    float64
cout                    float64
dtype: object


[226]